Note! This was not tested completely before submission. Small edits may be needed.

### Imports 

In [17]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas as pd
from gerrychain.tree import recursive_tree_part
from gerrychain.updaters import Tally, cut_edges
from gerrychain.metrics import efficiency_gap, mean_median, partisan_bias, polsby_popper, partisan_gini
import geopandas as gpd
import csv, json
get_ipython().run_line_magic('matplotlib', 'inline')

from tqdm import tqdm 

In [11]:
import os
os.getcwd()

'/home/ryan/Documents/LARedistrict'

### Read Input Files and Generate Graph

In [ ]:
laprec = geopandas.read_file("G:\My Drive\SFK Maps\Final Chain Data\laprecfinal.geojson")
laprec = laprec.to_crs('epsg:3452')

#laprec.to_file("G:\\My Drive\\SFK Maps\\Oct 21\\laprecfinal.geojson", driver='GeoJSON')
#laprec.to_csv("G:\My Drive\SFK Maps\Check of percmetrics\laprec.csv")

In [12]:
laprec = gpd.read_file("Gerry/laprecfinal.geojson")

In [25]:
# laprec.columns # Read Column Names laprec["Black_2020_VAP"]
laprec["Minority_2020_VAP"] = (laprec["Hispanic_2020_VAP"] + laprec["Black_2020_VAP"] +
                               laprec["Asian_2020_VAP"] + laprec["Native_2020_VAP"] 
                               laprec["Pacific_2020_VAP"])
laprec["NonMinority_2020_VAP"] = laprec["Total_2020_VAP"] - laprec["Minority_2020_VAP"]
laprec["NonDem_2016-2020_Comp"] = laprec["Total_2016-2020_Comp"] - laprec["Dem_2016-2020_Comp"]

In [ ]:
# Clean Up laprec

laprec = laprec[["Minority_2020_VAP", "NonMinority_2020_VAP", "NonDem_2016-2020_Comp"]]

#laprec.to_file("G:\\My Drive\\SFK Maps\\Oct 21\\laprecfinal.geojson", driver='GeoJSON')  

In [ ]:
# Create Graph
lp = Graph.from_geodataframe(laprec, ignore_errors=True)

### Constants

In [ ]:
SEN_DISTRICTS = 39
SEN_TARGET_POP = sum(laprec['TotalPop'])/39
POPULATION_COLUMN = "TotalPop"
TOLERANCE = .04
UNIQUE_LABEL = "GEOID20"
POP_COL = POPULATION_COLUMN
DISTRICT_COL = "sen_seed"
COUNTY_COL = "COUNTYFP10"

PASSIGN_FILE = f"G:\\My Drive\\SFK Maps\\Final Chain Data\\Senate\\Partitions\\partassgn{}.csv"

### Define Elections

In [ ]:
ElectionNames = ['2020_Pres', '2019_Gov', '2019_AG', '2019_Gov.1', '2016_Pres', '2016_Sen', '2016_Sen.1']
Elections = [Election(name, {'Democratic':f'Dem_{name}','Republican':f'Rep_{name}'}) for name in ElectionNames]
# num_elections = len(elections)
# election_columns = [[i.parties_to_columns['Democratic'], i.parties_to_columns['Republican']] for i in elections]

In [35]:
# We can apply Elections' built in percentages function to produce percentages for Demographics

DemoElections = [
    Election("Black_VAP_20", {'Black':'Black_2020_VAP','NonBlack':'NonBlack_2020_VAP'}),
    Election("Min_VAP_20", {'Minority':'Minority_2020_VAP','NonMinority':'NonBlack_2020_VAP'}),
    Election("Black_VAP_20", {'Black':'Black_2020_VAP','NonBlack':'NonBlack_2020_VAP'}),
    Election("Comp_Dems", {'Democrat':'Dem_2016-2020_Comp','NonDemocrat':'NonDem_2016-2020_Comp'}),
]




### Define Updaters

In [ ]:
def num_splits(partition): 
    laprec["current"] = laprec.index.map(dict(partition.assignment))
    splits = sum(laprec.groupby("COUNTYFP10")["current"].nunique() > 1)
    return splits

In [ ]:
# Population updater, for computing how close to equality the district
# populations are. "TOTPOP" is the population column from our shapefile.
my_updaters = {"population": updaters.Tally("TotalPop", alias="population"),
               "Total_2020_VAP": updaters.Tally("Total_2020_VAP"),
               
               # Sample Extra Other Updaters
               # "Total_2020_VAP": updaters.Tally("Total_2020_VAP"),
               # "Total_2016-2020_Comp": updaters.Tally("Total_2016-2020_Comp")   
              }

# Election updaters, for computing election results using the vote totals
# from our shapefile.

all_elections = Elections + DemoElections
all_updaters = {election.name: election for election in all_elections}
my_updaters.update(all_updaters)

### Initial Partition

In [ ]:
sen_partition = GeographicPartition(lp, assignment="sen_seed", updaters=my_updaters)

### Bounds and Constraints

In [ ]:
sencompactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(sen_partition["cut_edges"])
)

# Not needed as population is bounded by 'epsilon' in recom function?
senpop_constraint = constraints.within_percent_of_ideal_population(sen_partition, 0.05)

### Proposal and Chain

In [ ]:
senproposal = partial(recom,
                   pop_col="TotalPop",
                   pop_target=sen_target_pop,
                   epsilon=0.04,
                   node_repeats=2
                  )

senchain = MarkovChain(
    proposal=senproposal,
    constraints=[
        #? senpop_constraint,
        sencompactness_bound
    ],
    accept=accept.always_accept,
    initial_state=sen_partition,
    total_steps=50000
)

### Percmetrics Function

In [34]:
# Original Percmetrics Function

# # partnum is id of each new Partition 
# # part is each new partition 
# def percmetrics(partnum, part):
    
#     # global laprec
    
#     laprec["current"] = laprec.index.map(dict(part.assignment))
    
#     # Originally Returns pd.Series:
#     # distpop = laprec.groupby(["current"])["Total_2020_VAP"].sum()
#     # dempop = laprec.groupby(["current"])["Total_2016-2020_Comp"].sum()
    
#     distpop = pd.Series(laprec["Total_2020_VAP"])
#     dempop = pd.Series(laprec["Total_2016-2020_Comp"])

#     # dpdf = pandas.concat([distpop,dempop], axis=1)
#     # dpdf = dpdf.rename(columns={"Total_2020_VAP":"temppop", "Total_2016-2020_Comp":"dempop"})
    
#     laprec = laprec.merge(dpdf, how = "left", left_on="current", right_on = dpdf.index)
    
#     # "Weight Average", percent of prec VAP of assigned Dist VAP
#     laprec["wgtav"] = laprec["Total_2020_VAP"] / laprec["temppop"] 
    
#     # "Dem Weight Average", percent of prec Dem of assigned Dist Dems 
#     laprec['dwgt'] = laprec["Total_2016-2020_Comp"] / laprec["dempop"]
    
#     # "BlackPct" of Black VAP to Total VAP in Precinct
#     laprec["BlkPct"] = laprec["Black_2020_VAP"] /  laprec["Total_2020_VAP"]
    
#     # "MinPct" of Non-VAP to Total VAP in Precinct
#     # Note! Hispanics are counted in White_2020_VAP!
#     #       Should be sum of Hispanic, Black, Asian, Native, Pacific_2020_VAP
#     laprec["MinPct"] = (laprec["Total_2020_VAP"] - laprec["White_2020_VAP"] ) /  laprec["Total_2020_VAP"
    
#     # "DemPct" of Composite Dems vs Total Voters in Precinct
#     laprec["DemPct"] = laprec["Dem_2016-2020_Comp"] / laprec["Total_2016-2020_Comp"] 
    
#     # Produce weighted average for each Precinct                              
#     laprec["BlkPctW"] = laprec["BlkPct"] * laprec["wgtav"]
#     laprec["MinPctW"] = laprec["MinPct"] * laprec["wgtav"]
#     laprec["DemPctW"] = laprec["DemPct"] * laprec["dwgt"]
    
#     # Sum Each to Produce Percents
#     DistDemPct = laprec.groupby(["current"])["DemPctW"].sum()
#     DistMinPct = laprec.groupby(["current"])["MinPctW"].sum()
#     DistBlkPct = laprec.groupby(["current"])["BlkPctW"].sum()
    
#     DistDemPct = pandas.concat([DistDemPct, DistMinPct, DistBlkPct], axis=1)
    
#     DistDemPctD = DistDemPct.to_dict()
    
#     findict = {partnum: DistDemPctD}
    
#     del(laprec["temppop"])
#     del(laprec["dempop"])
    
#     # Produces dictionary where key is partnumer and value is dictionary with Dem, Min and Blk percentages.
#     return findict

## Chain-Run Block

In [37]:
def number_of_dists(percents, lothresh, hithresh):  
        perc_list = [True if (lothresh <= dist <= hithresh) else False for dist in percents]
        return sum(perc_list)

In [40]:
# Trackers

Mean_Median = pd.DataFrame(columns = ["ID", *ElectionNames]) # mms
Efficiency_Gap = pd.DataFrame(columns = ["ID", *ElectionNames]) # egs
Democratic_Wins = pd.DataFrame(columns = ["ID", *ElectionNames]) # hmss
Partisan_Gini = pd.DataFrame(columns = ["ID", *ElectionNames]) # pgs
Partisan_Bias = pd.DataFrame(columns = ["ID", *ElectionNames]) # pbs
# VOTES = pd.DataFrame...

trackercolumns = (["ID",
                   "splits", # splits
                   "dem_swings", # dct
                   "dem_maj", # dem
                   "maj_min", # mmct
                   "cut_vec"]) # Cut Edges 

Tracker = pd.DataFrame(columns = trackercolumns)

In [43]:
for partnum, part in enumerate(senchain.with_progress_bar()):

    tracker_dict = {"ID": partnum}
    # Splits
    
    splits = num_splits(part)
    # Count of Democratic Swing Dists
    dct = number_of_dists(part["Comp_Dems"].percents("Democrat"),.45,.55)
    # Count of Democratic Maj Dists
    dem = number_of_dists(part["Comp_Dems"].percents("Democrat"),.5,1.00)
    # Count of Minority Maj Dists
    mmct = number_of_dists(part["Min_VAP_20"].percents('Minority'),.45,.55)
    # Number of Cut Edges
    cut_vec = (len(part["cut_edges"]))
        
    tracker_dict.update({"splits": splits, "dem_swings": dct, "dem_maj": dem, "maj_min":mmct})
    Tracker.append(tracker_dict, ignore_index=True)
    
    if dem >= 9 and mmct >= 9 and dct >= 5:
        
        # Extract Partition Assignment Dict
        # Add GEOID20
        asgn_dict = dict(part.assignment)
        asgn_dict = {key:[asgn_dict[key], part.graph.nodes[key]["GEOID20"]]  for key in asgn_dict.keys()}
        
        # Create Partition Dataframe
        # Name Columns, Set Index
        # Save File
        asgn_df = pd.DataFrame.from_dict(  asgn_dict, orient='index').reset_index()
        asgn_df = asgn_df.rename({'index':'Node', 0:'Assignment', 1:'GEOID20'}, axis = 1).set_index("Node")
        sensdf.to_csv(PASSIGN_FILE.format(partnum)) 
    
        ## Update Trackers ##
        # One way to do it with dict comprehensions
        # mms = {election: mean_median(part[election]) for election in ElectionNames}.update({"ID": partnum})
        
        mms = {"ID": partnum}
        egs = {"ID": partnum}
        pgs = {"ID": partnum}
        pbs = {"ID": partnum}
        hmss = {"ID": partnum}
    
        for election in ElectionNames:
            # votes[elect].append(sorted(part[election_names[elect]].percents("Democratic")))
            mms.update({election: mean_median(part[election])})
            egs.update({election: efficiency_gap(part[election])})
            pgs.update({election: partisan_gini(part[election])})
            pbs.update({election: partisan_bias(part[election])})
            hmss.update({election: part[election].wins("Democratic")})
            
        Mean_Median.append(mms)
        Efficiency_Gap.append(egs)
        Democratic_Wins.append(pgs)
        Partisan_Gini.append(pbs)
        Partisan_Bias.append(hmss)
            
        print(partnum, mmct, dem, dct,"!!!!!")
    # else:
        # print(partnum, mmct,dem, dct)
           
Tracker.to_csv("G:\\My Drive\\SFK Maps\\Final Chain Data\\Senate\\Partitions\\tracker.txt")    

In [ ]:
# # Edited Original Chain-Run Block

# pop_vec = [] # sorted(list(part["population"].values()))
# cut_vec = [] # len(part["cut_edges"])
# votes = [[], [], [], [], [], [], [], [], [], [], [], [], [], []]
# mms = [] # mean_median
# egs = [] # efficiency_gap
# hmss = [] # part[election_names[elect]].wins("Democratic")
# pgs =[] # partisan_gini
# pbs = [] # partisan_bias
# splits = [] # num_splits
# pcents = [] # percmetrics accumulation

# f=  open("G:\\My Drive\\SFK Maps\\Final Chain Data\\Senate\\Partitions\\tracker.txt", "a+")

# # partnum is id of each new Partition 
# # part is each new partition 
# # use with_progress_bar() from tqdm 

# for partnum, part in enumerate(senchain.with_progress_bar()):
        
    
#     percd =  percmetrics(partnum,part)
    
#     dct = 0 # Count of Democratic Swing Dists on DemPctW
#     mmct = 0 # Count of Minority Maj Dists on MinPctW
#     dem = 0 # Count of Democratic Maj Dists on DemPctW
    
    
#     for key in percd[partnum]['DemPctW']:
#         if percd[partnum]['DemPctW'][key] >= 0.5:
#             dem += 1
#         if 0.45 <= percd[partnum]['DemPctW'][key] <= 0.55:
#             dct += 1
                
#     for key in percd[partnum]['MinPctW']:
#         if percd[partnum]['MinPctW'][key] >= 0.50:
#             mmct += 1
    
#     # If more than 9 majority Dem, 9 min maj dists and 5 swing dists
#     if dem >= 9 and mmct >= 9 and dct >= 5 :
#         pcents.append(percd) # add percmetrics to pcents
#         sens = [] # new list of sens
        
#         # Create Sens List for 
#         # for each node
#         g = list(part.graph.nodes) 
#         for y in g:
#             tt = [] # new tt list 
#             h = part.assignment[y] # h is assignment of node 
#             tt.append(y) # create [y, h]
#             tt.append(h)
#             sens.append(tt) # append [y, h] assignment list
        
        
#         # save sensdf to file
#         sensdf = pandas.DataFrame(sens)
#         sensdf.to_csv(PASSIGN_FILE.format(partnum)) 
    
#         splits.append(num_splits(part))
    
#         pop_vec.append(sorted(list(part["population"].values())))
#         cut_vec.append(len(part["cut_edges"]))
        
#         # create new arrays in these tracker lists
#         mms.append([])
#         egs.append([])
#         hmss.append([])
#         pgs.append([])
#         pbs.append([])
        
#         # for each election, add information 
#         for elect in range(num_elections):
#             votes[elect].append(sorted(part[election_names[elect]].percents("Democratic")))
#             mms[-1].append(mean_median(part[election_names[elect]]))
#             egs[-1].append(efficiency_gap(part[election_names[elect]]))
#             pgs[-1].append(partisan_gini(part[election_names[elect]]))
#             pbs[-1].append(partisan_bias(part[election_names[elect]]))
#             hmss[-1].append(part[election_names[elect]].wins("Democratic"))
#         print(partnum, mmct, dem, dct,"!!!!!")
#         f.write(str(partnum) +" "+ str(mmct) +" "+ str(dem) +" "+ str(dct)+" "+ "\n")
#     else:
#         print(partnum, mmct,dem, dct)
        
# f.close()

# # In[6]:

### Reporting

In [42]:
# Add suffix to column names
mmspd = Mean_Median.rename(columns = {c:f"{c}_mms" for c in Mean_median.columns})

# Etc...
egspd = Efficiency_Gap 
hmsspd = Democratic_Wins
pgspd = Partisan_Gini
pbspd = Partisan_Bias
spldpd = Tracker[["ID","Splits"]]


# mmspd = pandas.DataFrame(mms)
# egspd = pandas.DataFrame(egs)
# pgspd = pandas.DataFrame(pgs)
# pbspd = pandas.DataFrame(pbs)
# hmsspd = pandas.DataFrame(hmss)
# splpd = pandas.DataFrame(splits)
# compctpd = pandas.DataFrame(compct) # No compct in original

# mmspd = mmspd.rename(columns={0: "20Pres_mms",1: "1Gov_mms",2: "19AG_mms",3: "19Gov_1_mms",4: "16Pres_mms",5: "16sen_mms",6: "16_sen_1_mms"})
# pgspd = pgspd.rename(columns={0: "20Pres_pgs",1: "19Gov_pgs",2: "19AG_pgs",3: "19Gov_1_pgs",4: "16Pres_pgs",5: "16sen_pgs",6: "16_sen_1_pgs"})
# pbspd = pbspd.rename(columns={0: "20Pres_pbs",1: "19Gov_pbs",2: "19AG_pbs",3: "19Gov_1_pbs",4: "16Pres_pbs",5: "16sen_pbs",6: "16_sen_1_pbs"})
# egspd = mmspd.rename(columns={0: "20Pres_egs",1: "19Gov_egs",2: "19AG_egs",3: "19Gov_1_egs",4: "16Pres_egs",5: "16sen_egs",6: "16_sen_1_egs"})
# hmsspd = hmsspd.rename(columns={0: "20Pres_hmss",1: "19Gov_hmss",2: "19AG_hmss",3: "19Gov_1_hmss",4: "16Pres_hmss",5: "16sen_hmss",6: "16_sen_1_hmss"})
# splpd = splpd.rename(columns={0: "ParishSplits"})

# findf = pandas.concat([compctpd, mmspd,hmsspd,pgspd,pbspd,splpd],axis = 1)
# findf = findf.rename(columns={0: "ChainNum", 1: "CompDist",2: "DemMaj",3: "NonPckedBlck",4: "BlckMaj",5: "NonPckedMM",6: "MM"})

# #findf.to_csv("G:\My Drive\SFK Maps\Final Chain Data\lasen.csv")
# #laprec.to_csv("G:\My Drive\SFK Maps\Final Chain Data\laprec.csv")
# findf.to_csv("G:\\My Drive\\SFK Maps\\Final Chain Data\\Senate\\lasen.csv")

